<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Vectorization/NEWSBOMB_TRUMP_ARTICLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    # Check if the article_url is already a full URL
    if article_url_relative.startswith('http'):
        article_url = article_url_relative
    else:
        article_url = base_url + article_url_relative

    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trump_teasers_df = pd.read_csv("/content/drive/MyDrive/Εργασία Python/scrappers/TRUMP(24)-SCRAPER.CSV")
trump_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,Το νέο Air Force One του Τραμπ: Θα παραλάβει «...,https://www.newsbomb.gr/kosmos/story/1648636/t...,12.05.2025,07:56,ΚΟΣΜΟΣ,Ο Τραμπ πρόκειται να λάβει ένα «ιπτάμενο παλάτ...
1,Τραμπ: Προανήγγειλε εκτελεστικό διάταγμα για τ...,https://www.newsbomb.gr/kosmos/story/1648622/t...,12.05.2025,03:29,ΚΟΣΜΟΣ,Σκοπός να μειωθούν οι τιμές των συνταγογραφούμ...
2,Τραμπ: Μνημειώδης είδηση η συμφωνία απελευθέρω...,https://www.newsbomb.gr/kosmos/story/1648621/t...,12.05.2025,03:00,ΚΟΣΜΟΣ,«Είμαι ευγνώμων σε όλους όσοι συνέβαλαν να έρθ...
3,Ραντεβού στην Τουρκία δίνει ο Ζελένσκι - Θα πε...,https://www.newsbomb.gr/kosmos/story/1648569/r...,11.05.2025,20:22,ΚΟΣΜΟΣ,Η απάντηση ήρθε μετά τις αμερικανικές πιέσεις ...
4,Τραμπ: «Η Ουκρανία να συμφωνήσει άμεσα στην πρ...,https://www.newsbomb.gr/kosmos/story/1648556/t...,11.05.2025,19:31,ΚΟΣΜΟΣ,Νέα παρέμβαση του Αμερικανού προέδρου καθώς το...


In [ ]:
trump_teasers_df_test = trump_teasers_df.head(10)
trump_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Το νέο Air Force One του Τραμπ: Θα παραλάβει «...,https://www.newsbomb.gr/kosmos/story/1648636/t...,12.05.2025,07:56,ΚΟΣΜΟΣ,Ο Τραμπ πρόκειται να λάβει ένα «ιπτάμενο παλάτ...
1,Τραμπ: Προανήγγειλε εκτελεστικό διάταγμα για τ...,https://www.newsbomb.gr/kosmos/story/1648622/t...,12.05.2025,03:29,ΚΟΣΜΟΣ,Σκοπός να μειωθούν οι τιμές των συνταγογραφούμ...
2,Τραμπ: Μνημειώδης είδηση η συμφωνία απελευθέρω...,https://www.newsbomb.gr/kosmos/story/1648621/t...,12.05.2025,03:00,ΚΟΣΜΟΣ,«Είμαι ευγνώμων σε όλους όσοι συνέβαλαν να έρθ...
3,Ραντεβού στην Τουρκία δίνει ο Ζελένσκι - Θα πε...,https://www.newsbomb.gr/kosmos/story/1648569/r...,11.05.2025,20:22,ΚΟΣΜΟΣ,Η απάντηση ήρθε μετά τις αμερικανικές πιέσεις ...
4,Τραμπ: «Η Ουκρανία να συμφωνήσει άμεσα στην πρ...,https://www.newsbomb.gr/kosmos/story/1648556/t...,11.05.2025,19:31,ΚΟΣΜΟΣ,Νέα παρέμβαση του Αμερικανού προέδρου καθώς το...
5,Βασιλικό δώρο στον Ντόναλvτ Τραμπ - Το Κατάρ τ...,https://www.newsbomb.gr/kosmos/story/1648503/t...,11.05.2025,15:38,ΚΟΣΜΟΣ,Η αμερικανική κυβέρνηση πιθανότατα θα αποδεχθε...
6,Ο τελευταίος εν ζωή κρατούμενος του Αλκατράζ -...,https://www.newsbomb.gr/kosmos/story/1648483/a...,11.05.2025,13:14,ΚΟΣΜΟΣ,Ένας από τους τελευταίους εν ζωή κρατούμενους ...
7,Ινδία – Πακιστάν: Ο Τραμπ χαιρετίζει την «ιστο...,https://www.newsbomb.gr/kosmos/story/1648441/i...,11.05.2025,09:04,ΚΟΣΜΟΣ,«Αισθάνομαι υπερήφανος που οι Ηνωμένες Πολιτεί...
8,Η πρώτη αντίδραση Τραμπ στην πρόταση Πούτιν γι...,https://www.newsbomb.gr/kosmos/story/1648432/i...,11.05.2025,07:45,ΚΟΣΜΟΣ,Παράθυρο ελπίδας για τον πόλεμο στην Ουκρανία ...
9,Απομάκρυνση Γουόλτζ: Ο Τραμπ δημιουργεί ζώνη π...,https://www.newsbomb.gr/kosmos/story/1648429/a...,11.05.2025,07:27,ΚΟΣΜΟΣ,Η «σύγκρουση» μεταξύ της παλιάς φρουράς στη Γε...


In [ ]:
articles_df = scrape_newsbomb_articles_pages(trump_teasers_df)
articles_df

Processing rows: 100%|██████████| 2519/2519 [2:06:56<00:00,  3.02s/it]



Scraping completed. Fetshed 2519 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1648636/t...,Τραμπ: Θα παραλάβει «ιπτάμενο παλάτι» αξίας 40...,2025-05-12,07:56:02+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο Τραμπ πρόκειται να λάβει ένα «ιπτάμενο παλάτ...,Ο Λευκός Οίκος βρίσκεται σε συζητήσεις με τη β...,https://img.bbmd.gr/img/1260/max/82/2025/05/12...,O πρόεδρος Τραμπ κατεβαίνει από το Air Force O...
1,https://www.newsbomb.gr/kosmos/story/1648622/t...,Τραμπ: Προανήγγειλε εκτελεστικό διάταγμα για τ...,2025-05-12,03:29:07+03:00,ΚΟΣΜΟΣ,Newsbomb,Σκοπός να μειωθούν οι τιμές των συνταγογραφούμ...,ΟΝτόναλντ Τραμπανακοίνωσε χθες Κυριακή πως σήμ...,https://img.bbmd.gr/img/1260/max/82/2025/05/07...,Copyright 2025 The Associated Press. All right...
2,https://www.newsbomb.gr/kosmos/story/1648621/t...,Τραμπ: Μνημειώδης είδηση η συμφωνία απελευθέρω...,2025-05-12,03:00:56+03:00,ΚΟΣΜΟΣ,Newsbomb,«Είμαι ευγνώμων σε όλους όσοι συνέβαλαν να έρθ...,ΟΝτόναλντ Τραμπχαρακτήρισε χθες Κυριακή «μνημε...,https://img.bbmd.gr/img/1260/max/82/2025/05/11...,AP.
3,https://www.newsbomb.gr/kosmos/story/1648569/r...,Ραντεβού στην Τουρκία δίνει ο Ζελένσκι - Θα πε...,2025-05-11,20:22:40+03:00,ΚΟΣΜΟΣ,Newsbomb,Η απάντηση ήρθε μετά τις αμερικανικές πιέσεις ...,"Απέδωσαν οι πιέσεις του Αμερικανού προέδρου,Ντ...",https://img.bbmd.gr/img/1260/max/82/2025/05/11...,Αρχείου AP
4,https://www.newsbomb.gr/kosmos/story/1648556/t...,Τραμπ: «Η Ουκρανία να συμφωνήσει άμεσα στην πρ...,2025-05-11,19:31:21+03:00,ΚΟΣΜΟΣ,Newsbomb,Νέα παρέμβαση του Αμερικανού προέδρου καθώς το...,Μήνυμα στον Ουκρανό πρόεδρο έστειλε οΝτόναλντ ...,https://img.bbmd.gr/img/1260/max/82/2025/05/11...,
...,...,...,...,...,...,...,...,...,...,...
2514,https://www.newsbomb.gr/kosmos/story/1509543/t...,"Τραμπ: «Αν η Ευρώπη δεχτεί επίθεση, οι ΗΠΑ δεν...",2024-01-11,08:09:22+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Ντόναλντ Τραμπ είχε πει το 2020 στους Ευρωπα...,Ο Αμερικανόςπρώην πρόεδρος Ντόναλντ Τραμπείχε ...,https://img.bbmd.gr/img/1260/max/82/2024/01/11...,O Nτόναλντ Τραμπ\nΑΡ
2515,https://www.newsbomb.gr/kosmos/story/1509454/d...,Ντόναλτ Τραμπ: Μεγάλο προβάδισμα για το χρίσμα...,2024-01-10,20:33:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο πρώην πρόεδρος των ΗΠΑ είναι ο προτιμώμενος ...,Ενόψει των προεδρικών εκλογών στις ΗΠΑ μόλις π...,https://img.bbmd.gr/img/1260/max/82/2024/01/10...,Photo: AP
2516,https://www.newsbomb.gr/kosmos/story/1509430/i...,ΗΠΑ: Απεβίωσε η μητέρα της Μελάνια Τραμπ σε ηλ...,2024-01-10,18:59:02+02:00,ΚΟΣΜΟΣ,Newsbomb,"Η Αμαλία Κναβς, σλοβενικής υπηκοότητας, εγκατα...","Σε ηλικία 78 ετών απεβίωσε ηΑμαλία Κναβς, η μη...",https://img.bbmd.gr/img/1260/max/82/2024/01/10...,H μητέρα της Μελάνια Τραμπ Amalija Knavs\nPhot...
2517,https://www.newsbomb.gr/kosmos/story/1509191/m...,Μελάνια Τραμπ: Πέθανε η μητέρα της σε ηλικία 7...,2024-01-10,07:35:33+02:00,ΚΟΣΜΟΣ,Newsbomb,"Η Aμαλία Κναβς, μητέρα της Mελάνια Τραμπ, πέθα...",«Με βαθιά λύπη ανακοινώνω τον θάνατο της αγαπη...,https://img.bbmd.gr/img/1260/max/82/2024/01/10...,Πέθανε στα 78 της η πεθερά του Ντόναλντ Τραμπ...


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Ο Λευκός Οίκος βρίσκεται σε συζητήσεις με τη βασιλική οικογένεια του Κατάρ για την πιθανή παραλαβή ενός πολυτελούς τζάμπο τζετ, που προορίζεται να χρησιμοποιηθεί ωςπροεδρικό αεροσκάφος Air Force One. Σε ανακοίνωσή του,το Κατάρ αρνήθηκε ότι το αεροπλάνο θα είναι δώρο, αλλά δήλωσε ότι η μεταφορά ενός αεροσκάφους για «προσωρινή χρήση» βρίσκεται υπό συζήτηση μεταξύ των δύο χωρών. Ο 78χρονοςΝτόναλντ Τραμπσκοπεύει να χρησιμοποιήσει το αεροσκάφος - που θεωρείται το πιο πολυτελές ιδιωτικό τζετ στον κόσμο - ως προσωρινό προεδρικό τζετ, καθώς η Boeing ολοκληρώνει τον στόλο Air Force One επόμενης γενιάςμετά από χρόνια απογοητευτικών καθυστερήσεων και υπερβάσεων κόστους.Σύμφωνα με το CBS News, το τζάμπο τζετ, αξίας 400 εκατομμυρίων δολαρίων, θα μεταφερθεί ενδεχομένως στο ίδρυμα της προεδρικής βιβλιοθήκης του Τραμπ καθώς ο πρόεδρος αποχωρεί από το αξίωμα. Ο Αμερικανός πρόεδρος πρόκειται να επισκεφθεί το Κατάρ αυτή την εβδομάδαστο πλαίσιο του πρώτου μεγάλου ταξιδιού της δεύτερης θητείας του στο εξω

In [ ]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2025/05/12/8021d537-AP25125011272287.jpg?t=mtu_ol9zFu12qVQxXKXJrg'

In [ ]:
# prompt: βρες τα άρρθα από τις 20/11/2024 μέχρι τον απριλιο του 25

from datetime import datetime

# Convert the 'date' column to datetime objects
articles_df['date'] = pd.to_datetime(articles_df['date'], errors='coerce')

# Define the start and end dates
start_date = datetime(2024, 11, 20)
end_date = datetime(2025, 4, 30) # Assuming 'Απρίλιο του 25' means end of April 2025

# Filter the DataFrame
filtered_articles_df = articles_df[(articles_df['date'] >= start_date) & (articles_df['date'] <= end_date)].copy()

# Display the filtered DataFrame
print(f"\nFound {len(filtered_articles_df)} articles between {start_date.date()} and {end_date.date()}.")
filtered_articles_df


Found 1473 articles between 2024-11-20 and 2025-04-30.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
91,https://www.newsbomb.gr/kosmos/story/1645898/s...,«Σχέδιο Β» της Ευρώπης εάν ο Τραμπ εγκαταλείψε...,2025-04-30,22:43:56+03:00,ΚΟΣΜΟΣ,Newsbomb,"«Υπάρχουν εργαλεία στα χέρια των Αμερικανών, π...",ΗΕυρωπαϊκή Ένωσηετοιμάζει «σχέδιο Β» για τιςοι...,https://img.bbmd.gr/img/1260/max/82/2025/04/30...,AP.
92,https://www.newsbomb.gr/kosmos/story/1645874/t...,Τραμπ: Είμαστε έτοιμοι να υπογράψουμε και σήμε...,2025-04-30,21:06:03+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο πρόεδρος των Ηνωμένων Πολιτειών Ντόναλντ Τρα...,"Ο Ντόναλντ Τραμπ, απαντώντας σε ερώτηση δημοσι...",https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο πρόεδρος Ντόναλντ Τραμπ μιλάει κατά τη διάρκ...
93,https://www.newsbomb.gr/kosmos/story/1645864/t...,Τραμπ: «Η Κίνα τα πάει πολύ άσχημα αυτή τη στι...,2025-04-30,20:18:56+03:00,ΚΟΣΜΟΣ,Newsbomb,Έκθεση δείχνει συρρίκνωση της κινεζικής εργοστ...,Τη διαπίστωση ότι η οικονομία τηςΚίνας«πηγαίνε...,https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο πρόεδρος Ντόναλντ Τραμπ μιλάει κατά τη διάρκ...
94,https://www.newsbomb.gr/kosmos/story/1645860/i...,ΗΠΑ: Κινδυνεύει με απέλαση η Αμάλ Κλούνεΐ - Ο ...,2025-04-30,20:05:44+03:00,ΚΟΣΜΟΣ,Newsbomb,Πιθανή απαγόρευση εισόδου στις ΗΠΑ για τη γνωσ...,"Η Αμάλ Κλούνεΐ,δικηγόρος ανθρωπίνων δικαιωμάτω...",https://img.bbmd.gr/img/1260/max/82/2025/04/30...,Ο George Clooney και η σύζυγός του Amal Cloone...
95,https://www.newsbomb.gr/kosmos/story/1645828/s...,Συμφωνία ΗΠΑ-Ουκρανίας: Εμπόδια της τελευταίας...,2025-04-30,17:52:52+03:00,ΚΟΣΜΟΣ,Newsbomb,Αιφνίδιες διαφωνίες καθυστερούν την υπογραφή κ...,Ησυμφωνία Ηνωμένων Πολιτειών και Ουκρανίας για...,https://img.bbmd.gr/img/1260/max/82/2025/04/26...,AP Photo/Ben Curtis
...,...,...,...,...,...,...,...,...,...,...
1559,https://www.newsbomb.gr/kosmos/story/1603583/m...,Μαρκ Ρόουαν: Φαβορί για υπουργός Οικονομικών τ...,2024-11-20,10:27:19+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Mαρκ Ρόουαν αναδεικνύεται ως ο κορυφαίος υπο...,Ο δισεκατομμυριούχος ιδιωτικών επενδυτικών κεφ...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,
1560,https://www.newsbomb.gr/kosmos/story/1603570/h...,HΠΑ: Τι είναι ο ξέφρενος «χορός Τραμπ» και για...,2024-11-20,09:36:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ένας χορός που διαδόθηκε από τον Ντόναλντ Τραμ...,Ένας χορός που διαδόθηκε από τονεκλεγμένο πρόε...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,AP
1561,https://www.newsbomb.gr/kosmos/story/1603577/t...,Τουρκία προς Τραμπ: «Παράθυρο» για περιορισμέν...,2024-11-20,09:28:36+02:00,ΚΟΣΜΟΣ,Newsbomb,Η επιδίωξη της Άγκυρας αναφορικά με το εξοπλισ...,ΤοNewsbomb.grείχε γράψει από τον Αύγουστο για ...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,
1562,https://www.newsbomb.gr/kosmos/story/1603561/r...,Πούτιν: «Έτοιμος» να συζητήσει με τον Τραμπ κα...,2024-11-20,08:46:56+02:00,ΚΟΣΜΟΣ,Newsbomb,Ο Βλαντιμίρ Πούτιν είναι ανοιχτός στο ενδεχόμε...,Ο Βλαντιμίρ Πούτιν είναι ανοιχτός να συζητήσει...,https://img.bbmd.gr/img/1260/max/82/2024/11/20...,Associated Press


In [ ]:
duplicates_url = filtered_articles_df[filtered_articles_df.duplicated(subset=['article_url'], keep=False)]
print(f"\nArticles with duplicate URLs: {len(duplicates_url)}")
if not duplicates_url.empty:
    print(duplicates_url[['article_url', 'title']].sort_values(by='article_url'))

# Check for articles without title
no_title = filtered_articles_df[filtered_articles_df['title'].isnull() | (filtered_articles_df['title'].str.strip() == '')]
print(f"\nArticles without a title: {len(no_title)}")
if not no_title.empty:
    print(no_title[['article_url', 'title']])

# Check for articles without text body
no_text = filtered_articles_df[filtered_articles_df['text_body'].isnull() | (filtered_articles_df['text_body'].str.strip() == '')]
print(f"\nArticles without text body: {len(no_text)}")
if not no_text.empty:
    print(no_text[['article_url', 'title']])


Articles with duplicate URLs: 0

Articles without a title: 0

Articles without text body: 3
                                            article_url  \
1151  https://www.newsbomb.gr/kosmos/video/1622239/d...   
1282  https://www.newsbomb.gr/kosmos/video/1620247/g...   
1302  https://www.newsbomb.gr/kosmos/video/1620045/d...   

                                                  title  
1151  Ντόναλντ Τραμπ: Τι περιείχαν τα gift bags που ...  
1282  Γερουσιαστής «έσκασε μύτη» στην ορκωμοσία Τραμ...  
1302  Ντόναλντ Τραμπ: Χόρεψε ξανά συνοδεία του συγκρ...  


In [ ]:
trump_articles_25 = filtered_articles_df

In [ ]:
trump_articles_25 = trump_articles_25.drop([1151,1282,1302])

In [ ]:
from google.colab import drive

In [ ]:
scrape_newsbomb_articles_pages(trump_articles_25).to_csv('/content/drive/MyDrive/Εργασία Python/trump_articles_df_2025.csv2', index=False)

Processing rows: 100%|██████████| 1470/1470 [1:14:41<00:00,  3.05s/it]



Scraping completed. Fetshed 1470 articles.
